In [2]:
import numpy as np
import pandas as pd
import warnings
import os


In [3]:
df13 = pd.read_csv('aut_vincere_2013_trimmeddat.csv')
df14 = pd.read_csv('aut_vincere_2014_trimmeddat.csv')
df15 = pd.read_csv('aut_vincere_2015_trimmeddat.csv')
df16 = pd.read_csv('aut_vincere_2016_trimmeddat.csv')

### Establish connection to the postgres db:

In [5]:
import psycopg2
conn = psycopg2.connect("dbname=bball user=andric")
cur = conn.cursor()

Below two cells provide example for necessary "rollback" when twice a call is made on a bad query.

Postgres gives the `current transaction is aborted, commands ignored until of transaction block`
when this happens

In [21]:

# quer = "select * from playerdata WHERE season='2013';"
#cur.execute(quer)
# quer_out = cur.fetchall()
# pd.DataFrame(quer_out)

InternalError: current transaction is aborted, commands ignored until end of transaction block


In [24]:
# conn.rollback()

### This query grabs the column names from the db table

In [40]:
quer = "select column_name from information_schema.columns \
where table_schema = 'public' and table_name = 'playerdata';"
cur.execute(quer)
quer_out = cur.fetchall()
headers = [hh[0] for hh in quer_out]

### Now query to grab data.

In [43]:
# quer = "select * from playerdata where season='2013';"
# cur.execute(quer)
# data_quer_out = cur.fetchall()
# pd.DataFrame(data_quer_out).shape
# df13 = pd.DataFrame(data_quer_out, columns=headers).iloc[:, 1:]

(452, 29)

In [51]:
def get_season_data(year):
    """Get dataframe from query.
    Build list of them.
    """
    quer = "select * from playerdata where season='%s';" % year
    cur.execute(quer)
    data_quer_out = cur.fetchall()
    return data_quer_out


df_list = []
for yr in range(2013, 2017):
    df_list.append(pd.DataFrame(
            get_season_data(yr), columns=headers).iloc[:, 1:])

In [142]:
df13, df14, df15, df16 = df_list
players_intersect = list(set(df16.player).intersection(
        df15.player).intersection(
        df14.player).intersection(
        df13.player))

#### get players common to all four data frames:

In [143]:
def keepers(dataframe, player_list):
    return dataframe.iloc[np.where(np.in1d(dataframe.player,
                                           player_list)==True)[0], :]

In [167]:
keepers_df_list = []
for df in df_list:
    keepers_df_list.append(
        keepers(df, players_intersect).sort_values('player'))

df13, df14, df15, df16 = keepers_df_list

#### Now tables are sorted with same players.

In [171]:
df13

Index(['player', 'season', 'age', 'g', 'gs', 'mp', 'fg', 'fga', 'fgp', '3p',
       '3pa', '3pp', '2p', '2pa', '2pp', 'efgp', 'ft', 'fta', 'ftp', 'orb',
       'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'psg'],
      dtype='object')

In [158]:
df14.sort_values('player')

,player,season,age,g,gs,mp,fg,fga,fgp,3p,...,ftp,orb,drb,trb,ast,stl,blk,tov,pf,psg
414,Aaron Brooks,2014,29,48.000,8.000,22.433,3.333,8.400,0.401,1.333,...,0.872,0.600,1.400,2.000,3.433,0.733,0.167,0.980,2.067,9.300
171,Al Horford,2014,27,29.000,29.000,33.000,8.200,14.500,0.567,0.100,...,0.682,2.300,6.100,8.400,2.600,0.900,1.500,1.670,1.900,18.600
187,Al Jefferson,2014,29,73.000,73.000,35.000,9.600,18.800,0.509,0.000,...,0.690,2.100,8.700,10.800,2.100,0.900,1.100,1.000,2.400,21.800
7,Al-Farouq Aminu,2014,23,80.000,65.000,25.600,2.900,6.200,0.474,0.200,...,0.664,1.600,4.600,6.200,1.400,1.000,0.500,1.250,1.800,7.200
58,Alec Burks,2014,22,78.000,12.000,28.100,4.900,10.700,0.457,0.600,...,0.748,0.800,2.500,3.300,2.700,0.900,0.200,1.250,2.400,14.000
127,Alonzo Gee,2014,26,65.000,24.000,15.700,1.500,3.600,0.415,0.300,...,0.705,0.500,1.800,2.300,0.700,0.600,0.200,0.400,1.500,4.000
344,Amar'e Stoudemire,2014,31,65.000,21.000,22.600,4.800,8.600,0.000,0.000,...,0.000,1.700,3.200,4.900,0.500,0.400,0.600,0.000,2.400,11.900
368,Anderson Varejao,2014,31,65.000,29.000,27.700,3.500,7.100,0.495,0.000,...,0.681,2.900,6.800,9.700,2.200,1.100,0.600,1.250,2.500,8.400
176,Andre Iguodala,2014,30,63.000,63.000,32.400,3.500,7.300,0.480,1.000,...,0.652,0.800,3.800,4.700,4.200,1.500,0.300,1.430,1.600,9.300
444,Andre Miller,2014,37,38.667,1.333,16.867,1.800,3.967,0.459,0.200,...,0.786,0.467,1.767,2.200,3.400,0.600,0.167,1.447,1.300,4.867
